# kaak - Ein Actorframework wie akka, nur das fast alle Features fehlen  

Generatoren haben viel mit Aktoren gemeinsam:

* es können sehr viele Instanzen gleichzeitig existieren
* sie haben einen inneren State, der nicht geteilt wird
* man kann ihnen Nachrichten senden

Was nicht geht: 

* sie sind nicht reentrant

In [1]:
def ping():
    # msg ist unsere Message, der Ping actor schickt die Message einfach nur an den pong aktor
    while True:
        msg = yield # Auf send warten
        print("ping", msg)
        if msg == "stop":
            break
        send("pong", msg)

In [2]:
def pong():
    while True:
        msg = yield
        print("pong", msg)
        if msg < 10:
            send("ping", msg + 1)
        else:
            send("ping", "stop")

In [3]:
actors = {
    "ping": ping(),
    "pong": pong(),
}

In [4]:
# bevor wir senden können, müssen wir einmal next Aufrufen, damit der Generator bis zum yield laufen kann
for actor in actors.values():
    next(actor)

In [5]:
def send(actorname, msg):
    actors[actorname].send(msg)

In [6]:
send("ping", 0)

ping 0
pong 0


ValueError: generator already executing

## Es gibt kein Problem, was nicht durch eine weitere Stufe der Indirektion gelöst werden kann

Anstatt das sich die Generatoren selbst aufrufen, sollen sie ihre Nachricht in eine Queue schreiben.

Eine Messageloop holt dann immer einen Eintrag aus der Queue und sendet diese Nachricht.

In [7]:
from collections import deque

In [8]:
msgs = deque()

In [9]:
def send(actorname, msg):
    msgs.append((actorname, msg))

In [10]:
def run():
    while msgs:
        actorname, msg = msgs.popleft()
        actors[actorname].send(msg)

Die alten Generatoren laufen ja (pong) noch bzw. sind mit einer Exception abgestürzt (ping), daher erzeugen wir neue.

In [11]:
actors = {
    "ping": ping(),
    "pong": pong(),
}

In [12]:
# und lassen sie wieder bis zum yield laufen
for actor in actors.values():
    next(actor)

In [13]:
send("ping", 0)

In [14]:
run()

ping 0
pong 0
ping 1
pong 1
ping 2
pong 2
ping 3
pong 3
ping 4
pong 4
ping 5
pong 5
ping 6
pong 6
ping 7
pong 7
ping 8
pong 8
ping 9
pong 9
ping 10
pong 10
ping stop


StopIteration: 